# EAS 4610
## Final Group Project
## Glacial Cycles (Pollard 1982)

### Dennis Frank
### Marcus Morris
### Nathan Warner

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
import pandas as pd

# Introduction

# Equations

In [3]:
#Constants
a=0.81e-3 #Snowfall Constant, Unit:(1/yr) 
b=0.3e-6 #Ice Melt Constant, Unit:(1/(m*yr))
k=25 #Insolation Sensitivity
S=0 #Elevation of Current Sea Level
pi=917 #ice density (kg/m^3)
pw=1027 #sea water density (kg/m^3)
A = 5.77e-4 # 1/(m^3 yr)
alpha = 5
beta = 2
r = 0.3
nu = 100 # km^2/yr
nu = nu * 1000 * 1000

t_f = 700000
x_f = 4884000
dt = 50 #yr
dx = 55500 #m
nt = int(t_f/dt) + 1
nx = int(x_f/dx) + 1
x = np.linspace(0, x_f, nx)
t = np.linspace(0, t_f, nt)

c_nu = (nu * dt) / (dx**2)
c_A = (A * dt) / (4*dx**2)

In [4]:
#Function for Calculating the Mass Balance, G: (Snowfall - Ice Melt)
def G(h,h_prime,h_prev,h_prime_prev,E0,dQ): #h:ice height, h':bedrock elevation, x:distance, E0:current equilibrium altitude, dq:insolation change @ 65°N
    E = E0 + k*dQ
    H = h+h_prime #Surface Elevation: H=h+h'
    if (pi*h_prev < (pw*(S-h_prime_prev))) and (h_prime < S): #Calving Conditional Statement (maybe put this in separte fxn)
        return -20
    
    elif (H-E) <= 1500:
        return a*(H-E) - b*(H-E)**2
    
    else:
        return 0.56

In [6]:
h = np.zeros([nx, nt])
h_prime = np.zeros([nx, nt])

## Initial conditions
h_0 = np.zeros([nx])
h_prime_0 = np.zeros([nx])
h_naught = np.zeros([nx])

h[:, 0] = h_0
h_prime[:, 0] = h_prime_0

## EQUATION 3 MATRICES
h_prime_data = np.array([c_nu*np.ones(nx), (1-2*c_nu)*np.ones(nx), c_nu*np.ones(nx)])
h_prime_diags = np.array([-1, 0, 1])
h_prime_M_3 = spdiags(h_prime_data, h_prime_diags, nx, nx).toarray()

h_0_data = np.array([-c_nu*np.ones(nx), (2*c_nu)*np.ones(nx), -c_nu*np.ones(nx)])
h_0_diags = np.array([-1, 0, 1])
h_0_M_3 = spdiags(h_0_data, h_0_diags, nx, nx).toarray()

h_data = np.array([r*c_nu*np.ones(nx), (-2*r*c_nu)*np.ones(nx), r*c_nu*np.ones(nx)])
h_diags = np.array([-1, 0, 1])
h_M_3 = spdiags(h_data, h_diags, nx, nx).toarray()

## EQUATION 3 BOUNDARY CONDITIONS
h_prime_M_3[0, 0] = 0
h_prime_M_3[0, 1] = 0
h_prime_M_3[-1, -1] = 0
h_prime_M_3[-1, -2] = 0

h_0_M_3[0, 0] = 1
h_0_M_3[0, 1] = 0
h_0_M_3[-1, -1] = 1
h_0_M_3[-1, -2] = 0

h_M_3[0, 0] = 0
h_M_3[0, 1] = 0
h_M_3[-1, -1] = 0
h_M_3[-1, -2] = 0

#Define Insolation as a Function of Time (Q)
df = pd.read_csv('Milankovitch.dat',skiprows=1300,delim_whitespace=True, names=['year','lat','Q','x','y','z','v']) #only first 3 rows are of interest
Q = df['Q'].to_numpy() #values in W/m^2
n = np.ceil(len(t)/len(Q))
Q = list(np.repeat(Q,n)) #repeat values
#Run Loop to calculate dQ
Q_present=Q[-1]
dQ = np.ones((len(Q),1))
for i in np.arange(len(Q)):
    dQ[i] = Q[i] - Q_present
dQ = dQ.reshape(-1)
#Define Equilibrium-Line Altitude as a Function of 
E0 = ((0.9e-3)/111000)*x - (0.9e-3)

## MASTER LOOP
for k in np.arange(0, nt - 1):
    # EQUATION 1 MATRICES
    K = np.zeros([nx])
    K[1:-1] = h[1:-1,k]**alpha + abs(((h[2:nx,k] - h[0:nx-2,k])/(2*dx)) + ((h_prime[2:nx,k] - h_prime[0:nx-2,k])/(2*dx)))**beta
    K[0] = K[1]
    K[-1] = K[nx - 2]

    h_1 = np.zeros(nx)
    h_1[0:nx-2] = c_A*K[0:nx-2] + 4*c_A*K[1:nx-1] - c_A*K[2:nx]
    h_2 = np.zeros(nx)
    h_2[1:nx-1] = 1 - 8*c_A*K[1:nx-1]
    h_2[0] = 1
    h_2[-1] = 1
    h_3 = np.zeros(nx)
    h_3[2:nx] = -c_A*K[0:nx-2] + 4*c_A*K[1:nx-1] + c_A*K[2:nx]
    h_diags = np.array([-1, 0, 1])
    h_M = spdiags(np.array([h_1, h_2, h_3]), h_diags, nx, nx).toarray()

    h_p_1 = np.zeros(nx)
    h_p_1[0:nx-2] = c_A*K[0:nx-2] + 4*c_A*K[1:nx-1] - c_A*K[2:nx]
    h_p_2 = np.zeros(nx)
    h_p_2[1:nx-1] = -8*c_A*K[1:nx-1]
    h_p_3 = np.zeros(nx)
    h_p_3[2:nx] = -c_A*K[0:nx-2] + 4*c_A*K[1:nx-1] + c_A*K[2:nx]
    h_p_diags = np.array([-1, 0, 1])
    h_p_M = spdiags(np.array([h_p_1, h_p_2, h_p_3]), h_diags, nx, nx).toarray()
    
    # Find G
    G_M = np.zeros(nx)
    this_dQ = dQ[k]
    G_M[0] = G(h[0, k], h_prime[0, k], h[0, k], h_prime[0, k], E0[0], this_dQ)
    
    for i in np.arange(1, nx-1):
        G_M[i] = G(h[i, k], h_prime[i, k], h[i-1, k], h_prime[i-1, k], E0[i], this_dQ)
        
    # Find h at next time step
    h_new = h_M@h[:,k] + h_p_M@h_prime[:,k] + G_M
    h[:, k+1] = h_new.transpose()
    
    # Find h_prime at next time step    
    h_prime_new = h_prime_M_3@h_prime[:,k] + h_0_M_3@h_naught[:] + h_M_3@h[:,k]
    h_prime[:, k+1] = h_prime_new.transpose()
    
h

C:\Users\Dennis\AppData\Local\Temp\ipykernel_17292\2541280527.py:59: RuntimeWarning: overflow encountered in power
  K[1:-1] = h[1:-1,k]**alpha + abs(((h[2:nx,k] - h[0:nx-2,k])/(2*dx)) + ((h_prime[2:nx,k] - h_prime[0:nx-2,k])/(2*dx)))**beta
C:\Users\Dennis\AppData\Local\Temp\ipykernel_17292\2541280527.py:59: RuntimeWarning: overflow encountered in square
  K[1:-1] = h[1:-1,k]**alpha + abs(((h[2:nx,k] - h[0:nx-2,k])/(2*dx)) + ((h_prime[2:nx,k] - h_prime[0:nx-2,k])/(2*dx)))**beta
C:\Users\Dennis\AppData\Local\Temp\ipykernel_17292\2541280527.py:59: RuntimeWarning: invalid value encountered in add
  K[1:-1] = h[1:-1,k]**alpha + abs(((h[2:nx,k] - h[0:nx-2,k])/(2*dx)) + ((h_prime[2:nx,k] - h_prime[0:nx-2,k])/(2*dx)))**beta
C:\Users\Dennis\AppData\Local\Temp\ipykernel_17292\2541280527.py:64: RuntimeWarning: invalid value encountered in subtract
  h_1[0:nx-2] = c_A*K[0:nx-2] + 4*c_A*K[1:nx-1] - c_A*K[2:nx]
C:\Users\Dennis\AppData\Local\Temp\ipykernel_17292\2541280527.py:70: RuntimeWarning: inv

array([[ 0.00000000e+00,  7.28999757e-07, -4.75869419e-03, ...,
                    nan,             nan,             nan],
       [ 0.00000000e+00,  3.64499939e-07, -4.75942507e-03, ...,
                    nan,             nan,             nan],
       [ 0.00000000e+00,  8.78203760e-23, -4.76015595e-03, ...,
                    nan,             nan,             nan],
       ...,
       [ 0.00000000e+00, -3.06184287e-05, -4.82155071e-03, ...,
                    nan,             nan,             nan],
       [ 0.00000000e+00, -3.09829389e-05, -4.82228161e-03, ...,
                    nan,             nan,             nan],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
                    nan,             nan,             nan]])

# Numerical Methods

# Results

# Figures

# Video Walkthrough

# References